### 1.数据源
    1.特征数据要使用数组保存
    2.数据表必须有主键

    DROP TABLE IF EXISTS dt_golf_nb_train CASCADE;
    CREATE TABLE dt_golf_nb_train(
        id integer NOT NULL,
        data float8[],  ---不同于决策树,这里要使用数组
        class int
    );
    
    INSERT INTO dt_golf_nb_train VALUES (1,'{0,85,85,0}',0),
    (2, '{0,80,90,1}', 0),
    (3, '{1,83,78,0}', 1),
    (4, '{2,70,96,0}', 1),
    (5, '{2,68,80,0}', 1),
    (6, '{2,65,70,1}', 0),
    (7, '{1,64,65,1}', 1),
    (8, '{0,72,95,0}', 0),
    (9,  '{0,69,70,0}',1),
    (10, '{2,75,80,0}',1);

### 2.数据训练 
    http://madlib.apache.org/docs/latest/bayes_8sql__in.html#a3ed5ddbc4cab1013db8604be58a6da3d
    注意事项:
        要对数据源特征数据进行划分
        1.了解哪些是离散特征,哪些是连续特征,例如上面的特征2,3(温度,湿度)是连续的,1,4(天气,是否有风)是离散的
        2.连续特征 的 离散特征 调用的函数签名不一样,要区别对待. 
        3.调用函数后会生成3张表(模型表)

    select madlib.create_nb_prepared_data_tables(
        'dt_golf_nb_train', --源数据表
        'class',  --因变量的列名
        'data',  --自变量的列名
        'array[2,3]',--自变量中哪些字段是连续的,这里写上索引,如果自变量中没有连续字段,不要传入此参数
        4, --自变量的数量
        'dt_golf_nb_feature',--生成 表1 名
        'dt_golf_nb_num_feature',--生成 表2 名,如果自变量中没有连续字段,不要传入此参数
        'dt_golf_nb_class'----生成 表3 名
    );

### 3. 使用模型预测新数据

### 3.1 数据源
    DROP TABLE IF EXISTS dt_golf_nb_test CASCADE;
    CREATE TABLE dt_golf_nb_test(
        id integer NOT NULL,
        data float8[]  ---不同于决策树,这里要使用数组
    );
    INSERT INTO dt_golf_nb_test VALUES (1, '{0,75,70,1}'),
    (2, '{1,72,90,1}'),
    (3, '{1,72,90,1}'),
    (4, '{2,71,80,1}');

### 3.2 调用函数
    #可以得到预测 结果 和对于的概率
    select madlib.create_nb_probs_view(
        'dt_golf_nb_feature', --上面生成的表1
        'dt_golf_nb_class', --上面生成的表3
        'dt_golf_nb_test',  --输入表
        'id',               --主键名
        'data',             --自变量名
        4,                  --自变量数量
        'dt_golf_nb_num_feature',--上面生成的表2
        'dt_golf_nb_test_result_probs'
    );
    
    #只有预测结果,没有概率
    select madlib.create_nb_classify_view(
        'dt_golf_nb_feature', --上面生成的表1
        'dt_golf_nb_class', --上面生成的表3
        'dt_golf_nb_test',  --输入表
        'id',               --主键名
        'data',             --自变量名
        4,                  --自变量数量
        'dt_golf_nb_num_feature',--上面生成的表2
        'dt_golf_nb_test_result_cls'
    );